In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.special import logsumexp
from scipy.stats import norm
import random

# Set some defaults
plt.rc("axes.spines", top=False, right=False)

sns.set_theme(context="paper", font_scale=1.2)
sns.set_style("ticks")

# Import python library/function for function optimization
import scipy.optimize

%config InlineBackend.figure_format = "retina"

## The Plan

### Parameter and model recovery (Can I do both at same time?)
1) Simulate data using best-fit parameters from each model (or samples from within the range of best-fits). For example, 100 simulations of each model within range of best-fit parameters.
2) Re-estimate parameters.
3) Model comparison with AIC/BIC. 
4) Correlate parameters used for simulation with recovered parameters.


In [7]:
# Read in vmr data
df = pd.read_csv("../results/vmr_all.csv")

# Read in csv with MLEs from all models
# Need a function to convert string back to numpy array
def converter(input_str):
    return np.fromstring(input_str[1:-1], sep=' ')

fits = pd.read_csv("../results/params_mle.csv", converters={"theta":converter})
fits.head()


,subj_num,model,theta,loglik,bic,delta_bic
0,1,pea,"[7.49231206, 0.22499953]",-2997.492434,6009.209524,7.784242
1,1,premo,"[1.0, 2.73091915, 0.5, 4.63727082, 1.0]",-2982.931822,6001.425282,0.000000
2,1,rem,"[1.0, 17.73350857, 10.0, 1.14074609]",-2997.678305,6023.805920,22.380638
3,1,piece,"[5.64750593, 0.05, 4.05411942]",-2752.690198,5526.717378,-474.707904
4,2,pea,"[2.5712943, 0.57002164]",-2979.736177,5973.697008,-90.044514


In [6]:
# Lambda functions for computing negative log-likelihoods
nll_pea = lambda x: negloglik_test(model="pea", sigma_int=x[0], B=x[1],  
                                   sigma_motor=motor_sd, num_trials=len(subj), 
                                   vis_fb=vis_fb, rotation=rotation, x_hand=x_hand)
nll_premo = lambda x: negloglik_test(model="premo", B=x[0], sigma_pred=x[1], 
                                     sigma_v=x[2], sigma_p=x[3], eta_p=x[4], 
                                     sigma_motor=motor_sd, num_trials=len(subj), 
                                     vis_fb=vis_fb, rotation=rotation, 
                                     x_hand=x_hand)
nll_piece = lambda x: negloglik_test(model="piece", sigma_pert=x[0], sigma_pred=x[1], 
                                     sigma_p=x[2], sigma_motor=motor_sd, 
                                     num_trials=len(subj), vis_fb=vis_fb, 
                                     rotation=rotation, x_hand=x_hand)
nll_rem = lambda x: negloglik_test(model="rem", B=x[0], sigma_comb=x[1], s=x[2], c=x[3], 
                                   sigma_motor=motor_sd, num_trials=len(subj), 
                                   vis_fb=vis_fb, rotation=rotation, x_hand=x_hand)

In [ ]:
# Create data frame with all perturbation trials
df1 = pd.DataFrame({
    "rotation":subj.loc[pert_indices, "rotation"],
    "xhat_ssm":xhat_ssm[pert_indices],
    "adapt_ssm":adapt_ssm,
    "xhat_piece":xhat_piece[pert_indices],
    "adapt_piece":adapt_piece,
    "xhat_pea":xhat_pea[pert_indices],
    "adapt_pea":adapt_pea,
    "xhat_premo":xhat_premo[pert_indices],
    "adapt_premo":adapt_premo,
    "xhat_rem":xhat_rem[pert_indices], 
    "adapt_rem":adapt_rem
})


In [ ]:
models = ["piece", "pea", "premo", "rem"]
subj_num = []
model = []
winner = []

# Model recovery analysis
for i in range(len(np.unique(df["SN"]))):
    # Create dataframe with one subject's data
    sid = i + 1
    subj = df.loc[df["SN"] == sid, :].reset_index(drop=True)

    # Extract important variables
    motor_sd = subj.loc[0, "motor_sd"]
    x_hand = subj["theta_maxradv_clean"].values
    rotation = subj["rotation"].values
    vis_fb = subj["fbi"].values

    # Pull out best-fit params for chosen subject:
    params_idx = fits["subj_num"] == sid 
    params = fits.loc[params_idx, ["theta", "model"]].iloc[0:4, :].reset_index(drop=True)
    
    # Pull out MLEs for each model
    pea_mle = params.loc[params["model"] == "pea", "theta"].values[0]
    premo_mle = params.loc[params["model"] == "premo", "theta"].values[0]
    piece_mle = params.loc[params["model"] == "piece", "theta"].values[0]
    rem_mle = params.loc[params["model"] == "rem", "theta"].values[0]
    
    # Find perturbation trials
    pert_indices = np.flatnonzero(subj["perturbation"])

    # Loop through models
    for j in range(4):
        if j == 0:
            # Simulate with best-fit parameters
            # PIECE model
            _, xhat = piece(
                piece_mle[0], 
                piece_mle[1], 
                piece_mle[2], 
                sigma_motor, len(subj), 
                subj["fbi"], 
                subj["rotation"],
                fit=False
            )
            simulated_model = "piece"
        elif j == 1:
            # PEA model
            _, xhat = pea(
                pea_mle[0], 
                pea_mle[1], 
                sigma_motor, len(subj), 
                subj["fbi"], 
                subj["rotation"],
                fit=False
            )
            simulated_model = "pea"
        elif j == 2:
            # PReMo 
            _, xhat = premo(
                premo_mle[0],
                premo_mle[1],
                premo_mle[2], 
                premo_mle[3],
                premo_mle[4], 
                sigma_motor, len(subj), 
                subj["fbi"], 
                subj["rotation"],
                fit=False
            )
            simulated_model = "premo"
        elif j == 3:
            # REM
            _, xhat = rem(
                rem_mle[0],
                rem_mle[1],
                rem_mle[2], 
                rem_mle[3], 
                sigma_motor, len(subj), 
                subj["fbi"], 
                subj["rotation"],
                fit=False
            )
            simulated_model = "rem"
        # Assign simulated hand position to correct var name
        x_hand = xhat
        
        # Fit simulated data
        # PIECE model params: sigma_pert, sigma_pred, sigma_p
        bounds = ((0.05, 30), (0.05, 10), (0.05, 25))
        piece_results = scipy.optimize.minimize(
            fun=nll_piece, 
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1]),
                         np.random.uniform(low=bounds[2][0], high=bounds[2][1])])
        )
        bic_piece = calc_bic(piece_results.fun * -1, len(piece_results.x), len(subj))
        
        # PEA model params: sigma_comb, B
        bounds = ((0.5, 25), (0, 1))
        pea_results = scipy.optimize.minimize(
            fun=nll_pea, 
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1])])
        )
        bic_pea = calc_bic(pea_results.fun * -1, len(pea_results.x), len(subj))
        
        # PReMo model params: B, sigma_v, sigma_p, sigma_pred, eta_p
        bounds = ((0, 1), (0.05, 10), (0.5, 25), (0.5, 25), (0, 1))
        premo_results = scipy.optimize.minimize(
            fun=nll_premo,  
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1]),
                         np.random.uniform(low=bounds[2][0], high=bounds[2][1]),
                         np.random.uniform(low=bounds[3][0], high=bounds[3][1]),
                         np.random.uniform(low=bounds[4][0], high=bounds[4][1])])
        )
        bic_premo = calc_bic(premo_results.fun * -1, len(premo_results.x), len(subj))
        
        # REM model params: B, sigma_comb, s, c
        bounds = ((0, 1), (0.5, 25), (0, 10), (0, 10))
        rem_results = scipy.optimize.minimize(
            fun=nll_rem, 
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1]),
                         np.random.uniform(low=bounds[2][0], high=bounds[2][1]),
                         np.random.uniform(low=bounds[3][0], high=bounds[3][1])])
        )
        bic_rem = calc_bic(rem_results.fun * -1, len(rem_results.x), len(subj))

        # Store winning model
        winner.append(models[np.argmin([bic_piece, bic_pea, bic_premo, bic_rem])])

        # Update variables
        subj_num.append(sid)
        model.append(simulated_model)

df_model = pd.DataFrame({
    "subj_num":subj_num, 
    "model":model, 
    "winner":winner  
})